***Data loading rerec***

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
real_audio=[]
fake_audio=[]
# fake_audio_path = ['/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake','/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake','/kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/fake','/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake']
# real_audio_path = ['/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real','/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real','/kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/real','/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real']


fake_audio_path='/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake'
real_audio_path='/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real'
# for _dir in fake_audio_path:
for filename in os.listdir(fake_audio_path):
    if filename.endswith('.wav'):
        fake_audio.append(os.path.join(fake_audio_path, filename))
#         print(filename)
            
# for _dir in real_audio_path:
for filename in os.listdir(real_audio_path):
    if filename.endswith('.wav'):
        real_audio.append(os.path.join(real_audio_path, filename))

# real_audio_files = [os.path.join(real_audio_path, file) for file in os.listdir(real_audio_path) if file.endswith('.wav')]
# fake_audio_files = [os.path.join(fake_audio_path, file) for file in os.listdir(fake_audio_path) if file.endswith('.wav')]

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# For-rerec dataset

print("Number of tuples in fake_audio dataset: ", len(fake_audio))
# print(fake_audio[0])


print("Number of tuples in real_audio dataset: ", len(real_audio))
# print(real_audio[0])

In [ ]:
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import IPython
from librosa import display
from IPython import display as ipd

***For Rerecorded Audio***

> Visualise data

In [ ]:
# loading audio
ipd.Audio('/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake/recording1.wav_norm_mono.wav')

In [ ]:
real_ad, real_sr = librosa.load(real_audio[0])
plt.figure(figsize=(12, 4))
plt.plot(real_ad)
plt.title("Real Audio Data")
plt.show()

In [ ]:
fake_ad, fake_sr = librosa.load(fake_audio[0])
plt.figure(figsize=(12, 4))
plt.plot(fake_ad)
plt.title("Fake Audio Data")
plt.show()

In [ ]:
real_spec = np.abs(librosa.stft(real_ad))
real_spec = librosa.amplitude_to_db(real_spec, ref=np.max)

plt.figure(figsize=(10, 3))
librosa.display.specshow(real_spec, sr=real_sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")
plt.title("Real Audio Spectogram")
plt.show()

In [ ]:
fake_spec = np.abs(librosa.stft(fake_ad))
fake_spec = librosa.amplitude_to_db(fake_spec, ref=np.max)

plt.figure(figsize=(10, 3))
librosa.display.specshow(fake_spec, sr=fake_sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")
plt.title("Fake Audio Spectogram")
plt.show()

In [ ]:
# !pip install resampy

In [ ]:
# def feature_extraction(file_path):
#     x, sample_rate = librosa.load(file_path)
    
#     # Check if the loaded signal is empty
#     if len(x) == 0:
#         print(f"Warning: Empty audio signal in file {file_path}")
#         return None
    
#     mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=13).T, axis=0)
#     return mfcc

# real_audio_mfcc = []
# for audio in real_audio:
#     mfcc = feature_extraction(audio)
#     if mfcc is not None:
#         real_audio_mfcc.append(mfcc)


In [ ]:
# def feature_extraction(file_path):
#     x, sample_rate = librosa.load(file_path)
    
#     # Check if the loaded signal is empty
#     if len(x) == 0:
#         print(f"Warning: Empty audio signal in file {file_path}")
#         return None
    
#     mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=13).T, axis=0)
#     return mfcc

In [ ]:
# real_audio_mfcc = []
# for audio in real_audio:
#     mfcc = feature_extraction(audio)
# #     if mfcc is not None:
#     real_audio_mfcc.append(mfcc)

In [ ]:
# print(real_audio_mfcc)

fake data

In [ ]:
import os
folders = os.listdir(fake_audio_path)
# print(folders)

***preprocessing and feature extraction rerec***

In [ ]:
import os
data = []
labels = []

for file in tqdm(folders):
    file_path = os.path.join(fake_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    data.append(mfccs_features_scaled)
#     labels.append(folder)

***saving mfcc features in excel sheet***


In [ ]:
import pandas as pd
df = pd.DataFrame(data)

# Add column names (optional but recommended)
num_mfccs = data[0].shape[0]  # Assuming all have the same number of MFCCs
df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/fake_audio.xlsx'
df.to_excel(excel_file_path, index=False)
print(f"Features saved to: {excel_file_path}")

***Real Data***

In [ ]:
import os
realfolders = os.listdir(real_audio_path)
# print(folders)

***Preprocessing and feature Extraction of Real Data***


In [ ]:
import os
realdata = []
labels = []

for file in tqdm(realfolders):
    file_path = os.path.join(real_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    realdata.append(mfccs_features_scaled)
#     labels.append(folder)

***saving mfcc features in excel sheet***


In [ ]:
import pandas as pd
df = pd.DataFrame(realdata)

# Add column names (optional but recommended)
num_mfccs = realdata[0].shape[0]  # Assuming all have the same number of MFCCs
df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/real_re_audio.xlsx'
df.to_excel(excel_file_path, index=False)
print(f"Features saved to: {excel_file_path}")

In [ ]:
# fake_audio_mfcc = []
# for audio in fake_audio:
#     mfcc = feature_extraction(audio)
# #     if mfcc is not None:
#     fake_audio_mfcc.append(mfcc)

In [ ]:
# fnum_mfccs = fake_audio_mfcc[0].shape[0]  # Assuming all have the same number of MFCCs
# df.columns = [f'MFCC_{i}' for i in range(fnum_mfccs)]

# # Save DataFrame to Excel
# excel_file_path = '/kaggle/working/rerec_fake_audio.xlsx'
# df.to_excel(excel_file_path, index=False)
# print(f"Features saved to: {excel_file_path}")

In [ ]:
!ls /kaggle/working

In [ ]:
# print(data)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
real_2sec_audio=[]
fake_2sec_audio=[]
# fake_audio_path = ['/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake','/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/fake','/kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/fake','/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/fake']
# real_audio_path = ['/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real','/kaggle/input/the-fake-or-real-dataset/for-norm/for-norm/training/real','/kaggle/input/the-fake-or-real-dataset/for-original/for-original/training/real','/kaggle/input/the-fake-or-real-dataset/for-rerec/for-rerecorded/training/real']


fake_2sec_audio_path='/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake'
real_2sec_audio_path='/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/real'
# for _dir in fake_audio_path:
for filename in os.listdir(fake_2sec_audio_path):
#     if filename.endswith('.wav'):
    fake_2sec_audio.append(os.path.join(fake_2sec_audio_path, filename))
#         print(filename)
            
# for _dir in real_audio_path:
for filename in os.listdir(real_2sec_audio_path):
#     if filename.endswith('.wav'):
    real_2sec_audio.append(os.path.join(real_2sec_audio_path, filename))

# real_audio_files = [os.path.join(real_audio_path, file) for file in os.listdir(real_audio_path) if file.endswith('.wav')]
# fake_audio_files = [os.path.join(fake_audio_path, file) for file in os.listdir(fake_audio_path) if file.endswith('.wav')]

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# For-2sec dataset

print("Number of tuples in fake_audio dataset: ", len(fake_2sec_audio))
# print(fake_2sec_audio[0])


print("Number of tuples in real_audio dataset: ", len(real_2sec_audio))
# print(real_2sec_audio[0])


In [ ]:
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import IPython
from librosa import display
from IPython import display as ipd

In [ ]:
import IPython.display as ipd
file_path = '/kaggle/input/the-fake-or-real-dataset/for-2sec/for-2seconds/training/fake/file10005.mp3.wav_16k.wav_norm.wav_mono.wav_silence.wav_2sec.wav'

# 20000 for 20 kHz audio
sample_rate = 20000
ipd.Audio(file_path, rate=sample_rate)


In [ ]:
real_ad, real_sr = librosa.load(real_2sec_audio[0])
plt.figure(figsize=(12, 4))
plt.plot(real_ad)
plt.title("Real Audio Data")
plt.show()

In [ ]:
fake_ad, fake_sr = librosa.load(fake_2sec_audio[0])
plt.figure(figsize=(12, 4))
plt.plot(fake_ad)
plt.title("Fake Audio Data")
plt.show()

In [ ]:
real_spec = np.abs(librosa.stft(real_ad))
real_spec = librosa.amplitude_to_db(real_spec, ref=np.max)

plt.figure(figsize=(10, 3))
librosa.display.specshow(real_spec, sr=real_sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")
plt.title("Real Audio Spectogram")
plt.show()

In [ ]:
fake_spec = np.abs(librosa.stft(fake_ad))
fake_spec = librosa.amplitude_to_db(fake_spec, ref=np.max)

plt.figure(figsize=(10, 3))
librosa.display.specshow(fake_spec, sr=fake_sr, x_axis="time", y_axis="log")
plt.colorbar(format="%+2.0f dB")
plt.title("Fake Audio Spectogram")
plt.show()

***Fake 2sec data***

In [ ]:
import os
folders_fake_2sec=os.listdir(fake_2sec_audio_path)
# print(folders_fake_2sec)

***Preprocessing and Feature Extraction of fake 2 sec data***

In [ ]:
import os
data_fake_2sec = []
labels = []

for file in tqdm(folders_fake_2sec):
    file_path = os.path.join(fake_2sec_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    data_fake_2sec.append(mfccs_features_scaled)
#     labels.append(folder)

***Storing the data of fake 2 sec data in excel sheet***

In [ ]:
import pandas as pd
df = pd.DataFrame(data_fake_2sec)

# Add column names (optional but recommended)
num_mfccs = data_fake_2sec[0].shape[0]  # Assuming all have the same number of MFCCs
df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/fake_2sec_audio.xlsx'
df.to_excel(excel_file_path, index=False)
print(f"Features saved to: {excel_file_path}")

***Real 2 sec data***

In [ ]:
import os
folders_real_2sec = os.listdir(real_2sec_audio_path)
# print(folders_real_2sec)

***Preprocessing and feature extraction of real 2 sec data***

In [ ]:
import os
data_real_2sec = []
labels = []

for file in tqdm(folders_real_2sec):
    file_path = os.path.join(real_2sec_audio_path, file)
    audio, sample_rate = librosa.load(file_path)        
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfccs_features.T, axis=0)
    data_real_2sec.append(mfccs_features_scaled)
#     labels.append(folder)

***Storing the real 2 sec data in excel sheet***

In [ ]:
import pandas as pd
df = pd.DataFrame(data_real_2sec)

# Add column names (optional but recommended)
num_mfccs = data_real_2sec[0].shape[0]  # Assuming all have the same number of MFCCs
df.columns = [f'MFCC_{i}' for i in range(num_mfccs)]

# Save DataFrame to Excel
excel_file_path = '/kaggle/working/real_2sec_audio.xlsx'
df.to_excel(excel_file_path, index=False)
print(f"Features saved to: {excel_file_path}")

In [ ]:
!ls /kaggle/working